In [1]:
import pandas as pd
import pickle as pkl

# Global variables

In [2]:
data_folder_name = 'PPG_FieldStudy'

# Functions

In [3]:
def get_script_dir_path():
    script_dir_path = %pwd 
    script_dir_path += '\\'
    return script_dir_path

# 1) Data preparation

In [4]:
script_dir_path = get_script_dir_path()
print(script_dir_path)

C:\Users\Louis\python_scripts\final_project\python_for_data_analysis\


In [5]:
data_folder_path = script_dir_path + data_folder_name + '\\'
print(data_folder_path)

C:\Users\Louis\python_scripts\final_project\python_for_data_analysis\PPG_FieldStudy\


In [14]:
# Creating path to pkl file for each subject
pkl_path_template = "S{}\\S{}.pkl"
pkl_pd_list = []
for i in range(15):
    path = data_folder_path + pkl_path_template.format(i+1,i+1)
    print(path)
    pkl_pd_list.append(pd.read_pickle(path))

C:\Users\Louis\python_scripts\final_project\python_for_data_analysis\PPG_FieldStudy\S1\S1.pkl
C:\Users\Louis\python_scripts\final_project\python_for_data_analysis\PPG_FieldStudy\S2\S2.pkl
C:\Users\Louis\python_scripts\final_project\python_for_data_analysis\PPG_FieldStudy\S3\S3.pkl
C:\Users\Louis\python_scripts\final_project\python_for_data_analysis\PPG_FieldStudy\S4\S4.pkl
C:\Users\Louis\python_scripts\final_project\python_for_data_analysis\PPG_FieldStudy\S5\S5.pkl
C:\Users\Louis\python_scripts\final_project\python_for_data_analysis\PPG_FieldStudy\S6\S6.pkl
C:\Users\Louis\python_scripts\final_project\python_for_data_analysis\PPG_FieldStudy\S7\S7.pkl
C:\Users\Louis\python_scripts\final_project\python_for_data_analysis\PPG_FieldStudy\S8\S8.pkl
C:\Users\Louis\python_scripts\final_project\python_for_data_analysis\PPG_FieldStudy\S9\S9.pkl
C:\Users\Louis\python_scripts\final_project\python_for_data_analysis\PPG_FieldStudy\S10\S10.pkl
C:\Users\Louis\python_scripts\final_project\python_for_dat

In [16]:
print(pkl_pd_list[0]['signal']['chest']['ACC'])

[[ 0.85640001 -0.06779999 -0.36559999]
 [ 0.8556     -0.06900001 -0.3646    ]
 [ 0.852      -0.07020003 -0.3642    ]
 ...
 [ 0.88759995 -0.1038     -0.27920002]
 [ 0.88619995 -0.10159999 -0.27819997]
 [ 0.88680005 -0.10140002 -0.27380002]]


In [29]:
i = 0
# Capteur Empatica E4
wrist_ACC = pkl_pd_list[i]['signal']['wrist']['ACC']
wrist_BVP = pkl_pd_list[i]['signal']['wrist']['BVP']
wrist_EDA = pkl_pd_list[i]['signal']['wrist']['EDA']
wrist_TEMP = pkl_pd_list[i]['signal']['wrist']['TEMP']

# Capteur RespiBAN
chest_ACC = pkl_pd_list[i]['signal']['chest']['ACC']
chest_ECG = pkl_pd_list[i]['signal']['chest']['ECG']
chest_resp = pkl_pd_list[i]['signal']['chest']['Resp']

# Activity
activity = pkl_pd_list[i]['activity']


[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
